DSCI Final Project - Baseball Statistics

MLB Stats API: <br>
https://pypi.org/project/MLB-StatsAPI/ <br>
Wiki/Documentation: https://github.com/toddrob99/MLB-StatsAPI/wiki <br>
Usage: run 'pip install MLB-StatsAPI' <br><br>
Datasets:<br>
SFBB-Player-ID-Map.csv: from https://www.smartfantasybaseball.com/tools/ <br>
expected_stats.csv: from https://baseballsavant.mlb.com/leaderboard/expected_statistics?type=batter&year=2015&position=&team=&min=q <br>
Batting.csv: from https://www.kaggle.com/datasets/open-source-sports/baseball-databank?select=Batting.csv


In [49]:
# Setups
import pandas as pd
import plotly.express as px
import statsapi
#import logging

In [50]:
data_location = "datasets/"

exp_stats = pd.read_csv(data_location + "expected_stats.csv")
batting_names = pd.read_csv(data_location + 'SFBB-Player-ID-Map.csv')
batting = pd.read_csv(data_location + 'Batting.csv')

In [51]:
#renames columns of exp_stats and makes it easier to view
exp_stats = exp_stats.rename(columns={'last_name':'Name',
                                      ' first_name':'to_drop',
                                      'player_id':'ID',
                                      'year':'Year',
                                      'pa':'Plate Appearances',
                                      'bip':'Balls In Play',
                                      'ba':'BA',
                                      'est_ba':'xBA',
                                      'est_ba_minus_ba_diff':'xBA - BA Diff',
                                      'slg':'Slg',
                                      'est_slg':'xSlg',
                                      'est_slg_minus_slg_diff':'xSlg - Slg Diff',
                                      'woba':'WOBA',
                                      'est_woba':'xWOBA',
                                      'est_woba_minus_woba_diff':'xWOBA - WOBA Diff'})
exp_stats['Name'] = exp_stats['to_drop'] + ' ' + exp_stats['Name']
exp_stats = exp_stats.drop(columns =['to_drop','Year'])
#exp_stats = exp_stats.set_index('Name')

In [52]:
#renames columns of batting and makes it easier to view
batting = batting[batting.yearID == 2015]
batting = batting.drop(columns=['yearID', 'stint', 'lgID'])
batting = batting.rename(columns = {'playerID':'Player ID',
                                    'teamID':'Team',
                                    'G':'Games'})
#batting = batting.set_index('Player ID')

In [53]:
batting_final = pd.merge(batting_names[['IDPLAYER','PLAYERNAME', 'BATS']], batting,  right_on='Player ID', left_on='IDPLAYER')
batting_final = batting_final.rename(columns = {'PLAYERNAME':'Name'})
batting_final = batting_final.set_index('Name')

In [54]:
display(batting_final)
display(exp_stats)

,IDPLAYER,BATS,Player ID,Team,Games,AB,R,H,2B,3B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
Name,,,,,,,,,,,,,,,,,,,,,
David Aardsma,aardsda01,R,aardsda01,ATL,33,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Fernando Abad,abadfe01,L,abadfe01,OAK,62,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jose Abreu,abreujo02,R,abreujo02,CHA,154,613.0,88.0,178.0,34.0,3.0,...,101.0,0.0,0.0,39.0,140.0,11.0,15.0,0.0,1.0,16.0
Dustin Ackley,ackledu01,L,ackledu01,SEA,85,186.0,22.0,40.0,8.0,1.0,...,19.0,2.0,2.0,14.0,38.0,0.0,1.0,3.0,3.0,3.0
Dustin Ackley,ackledu01,L,ackledu01,NYA,23,52.0,6.0,15.0,3.0,2.0,...,11.0,0.0,0.0,4.0,7.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Barry Zito,zitoba01,L,zitoba01,OAK,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ben Zobrist,zobribe01,B,zobribe01,OAK,67,235.0,39.0,63.0,20.0,2.0,...,33.0,1.0,1.0,33.0,26.0,2.0,0.0,0.0,3.0,5.0
Ben Zobrist,zobribe01,B,zobribe01,KCA,59,232.0,37.0,66.0,16.0,1.0,...,23.0,2.0,3.0,29.0,30.0,1.0,1.0,0.0,2.0,3.0


,Name,ID,Plate Appearances,Balls In Play,BA,xBA,xBA - BA Diff,Slg,xSlg,xSlg - Slg Diff,WOBA,xWOBA,xWOBA - WOBA Diff
0,Marcus Semien,543760,724,547,0.248,0.243,0.005,0.429,0.394,0.035,0.317,0.306,0.011
1,Freddie Freeman,518692,708,517,0.325,0.313,0.012,0.511,0.538,-0.027,0.393,0.403,-0.010
2,Trea Turner,607208,708,527,0.298,0.276,0.022,0.466,0.432,0.034,0.350,0.335,0.015
3,Vladimir Guerrero Jr.,665489,706,526,0.274,0.276,-0.002,0.480,0.462,0.018,0.351,0.348,0.003
4,Francisco Lindor,596019,706,504,0.270,0.254,0.016,0.449,0.427,0.022,0.342,0.331,0.011
...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,Gilberto Celestino,665482,347,236,0.238,0.219,0.019,0.302,0.294,0.008,0.280,0.270,0.010
248,Lars Nootbaar,663457,347,224,0.228,0.247,-0.019,0.448,0.414,0.034,0.342,0.346,-0.004
249,Wander Franco,677551,344,285,0.277,0.285,-0.008,0.417,0.408,0.009,0.323,0.331,-0.008
250,Jesús Sánchez,660821,343,222,0.214,0.226,-0.012,0.403,0.383,0.020,0.298,0.298,0.000


In [55]:
#print( statsapi.standings(leagueId=103,date='05/01/2023') )
#print( statsapi.player_stats(next(x['id'] for x in statsapi.get('sports_players',{'season':2018,'gameType':'W'})['people'] if x['fullName']=='Juan Soto'), 'hitting', 'season') )
with open('test_current_players.txt', 'w') as f:
    #for player in exp_stats['Name']:
    f.write(statsapi.player_stats(next(x['id'] for x in statsapi.get('sports_players',{'season':2018,'gameType':'W'})['people'] if x['fullName']=='Juan Soto'), 'hitting', 'season'))